# Calculate Metrics

**Description:** This notebook compares different groups of experiments, using the metrics described in each cell.

**Real Time**
* Burst duration
* Rise duration
* Decay duration
* Interval duration
* SNA duration

**Developmental Time**
* At Onset
* At Hatching
* Per Burst
* CDF

**Other Features**
* Amplitude
* AUC Bins
* Local Peaks

## Import Libraries

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import pandas as pd

from pasna_analysis import Experiment, ExperimentConfig, Group, utils, myplots

## Load Data

In [ ]:
wt_folder = '25C'
wt_config  = ['20240611_25C', '20240919_25C']

rdl_folder = 'rdl'
rdl_config = ['20250211_rdl','20250212_rdl']

wt_experiments = {}
for exp in wt_config:
    exp_path = Path.cwd().parent.joinpath('data', wt_folder, exp)
    wt_experiments[exp] = Experiment(exp_path)

rdl_experiments = {}
for exp in rdl_config:
    exp_path = Path.cwd().parent.joinpath('data', rdl_folder, exp)
    rdl_experiments[exp] = Experiment(exp_path)

wt = Group('WT', wt_experiments)
rdl = Group('Rdl-', rdl_experiments)

groups = [wt, rdl]

## Configure Plot Style

Customize the Matplotlib plots with rcParams. For a full list of parameters see [matplotlib.rcParams](https://matplotlib.org/stable/api/matplotlib_configuration_api.html#default-values-and-styling)

In [ ]:
myplots.set_darkbg(False)

num_episodes = 15

episode_metrics_rc = {
    'figure.figsize': (6, 3),
    'figure.frameon': False,

    'lines.linewidth': 1,

    'font.family': 'Arial',
    'font.size': 20,

    'xtick.major.size': 10, # length
    'xtick.major.width': 2,
    'ytick.major.size': 10, # length
    'ytick.major.width': 2,

    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.labelpad': 10,
    'axes.linewidth': 2.5
}

num_groups = len(groups)
group_metrics_rc = {
    'figure.figsize': (6, 3),
    'figure.frameon': False,

    'lines.linewidth': 2.25,

    'font.family': 'Arial',
    'font.size': 20,

    'xtick.major.size': 10, # length
    'xtick.major.width': 2,
    'ytick.major.size': 10, # length
    'ytick.major.width': 2,

    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.labelpad': 10,
    'axes.linewidth': 2.5
}


# Overview

## Sample Size

Prints a table displaying the total sample size for each group, plus the sample size for each experiment in a group. 

In [ ]:
data = {"Group":[], "Exp":[], "Emb":[]}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            data["Group"].append(group.name)
            data["Exp"].append(exp.name)
            data["Emb"].append(emb)

data = pd.DataFrame(data)

# calculate total Ns per group
num_per_exp = data.groupby(["Group", "Exp"]).size().reset_index(name="N")
num_per_group = data.groupby("Group").size().reset_index(name="N")
num_per_group["Exp"] = "**Total**"
num_per_group = num_per_group[["Group", "Exp", "N"]]
combined = pd.concat([num_per_exp, num_per_group], ignore_index=True)
flipped = combined[::-1]

print(flipped.to_string(index=False))

# Real Time

## Burst Duration

In [ ]:
data = {"Group": [], "Burst Duration (min)": [], "Burst #": []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos.values():
            trace = emb.trace
            for i, duration in zip(range(num_episodes), trace.peak_durations):
                data["Group"].append(group.name)
                data["Burst Duration (min)"].append(duration / 60)
                data["Burst #"].append(i+1)

data = pd.DataFrame(data)

myplots.plot_pointplot(data, x="Burst #", y="Burst Duration (min)", category="Group", rc=episode_metrics_rc, linestyle='-.')

## Rise Duration

In [ ]:
data = {"Group": [], "Rise Duration (sec)": [], "Burst #": []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos.values():
            trace = emb.trace
            for i, rise in zip(range(num_episodes), trace.peak_rise_times):
                data["Group"].append(group.name)
                data["Rise Duration (sec)"].append(rise)
                data["Burst #"].append(i+1)

data = pd.DataFrame(data)

myplots.plot_pointplot(data, x="Burst #", y="Rise Duration (sec)", category="Group", rc=episode_metrics_rc, linestyle='-.')

## Decay Duration

In [ ]:
data = {"Group": [], "Decay Duration (sec)": [], "Burst #": []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos.values():
            trace = emb.trace
            for i, decay in zip(range(num_episodes), trace.peak_decay_times):
                data["Group"].append(group.name)
                data["Decay Duration (sec)"].append(decay)
                data["Burst #"].append(i+1)

data = pd.DataFrame(data)

myplots.plot_pointplot(data, x="Burst #", y="Decay Duration (sec)", category="Group", rc=episode_metrics_rc, linestyle='-.')

## Interval Duration

In [ ]:
data = {"Group": [], "Interval Duration (min)": [], "Burst #": []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos.values():
            trace = emb.trace
            for i, interval in zip(range(num_episodes), trace.peak_intervals):
                data["Group"].append(group.name)
                data["Interval Duration (min)"].append(interval / 60)
                data["Burst #"].append(i+1)

data = pd.DataFrame(data)

myplots.plot_pointplot(data, x="Burst #", y="Interval Duration (min)", category="Group", rc=episode_metrics_rc, linestyle='-.')

## SNA Duration

In [ ]:
data = {"Group": [], "Duration (min)": []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos.values():
            trace = emb.trace
            data["Group"].append(group.name)
            duration = (trace.time[trace.trim_idx] - trace.peak_times[0]) / 60
            data["Duration (min)"].append(duration)

data = pd.DataFrame(data)

myplots.plot_pointplot(data, x="Group", y="Duration (min)", category="Group", rc=episode_metrics_rc, xmax=len(groups)-1)

# Developmental Time

## Dev Time at Onset

In [ ]:
data = {"Group": [], "Dev Time": []}

for group in groups:
    for exp in group.experiments.values():
        num = 0
        for emb in exp.embryos.values():
            num = num + 1
            trace = emb.trace
            time_first_peak = trace.peak_times[0]
            dev_time_first_peak = emb.get_DT_from_time(time_first_peak)
            data["Dev Time"].append(dev_time_first_peak)
            data["Group"].append(group.name)

data = pd.DataFrame(data)

myplots.plot_pointplot(data, x="Group", y="Dev Time", category="Group", rc=group_metrics_rc, xmax=len(groups)-1)

## Dev Time at Hatching

In [ ]:
data = {"Group": [], "Dev Time": []}

for group in groups:
    for exp in group.experiments.values():
        num = 0
        for emb in exp.embryos.values():
            num = num + 1
            trace = emb.trace
            time_hatching = trace.time[trace.trim_idx]
            dev_time_first_peak = emb.get_DT_from_time(time_hatching)

            data["Dev Time"].append(dev_time_first_peak)
            data["Group"].append(group.name)

data = pd.DataFrame(data)

myplots.plot_pointplot(data, x="Group", y="Dev Time", category="Group", rc=group_metrics_rc, xmax=len(groups)-1)

## Dev Time Per Burst

In [ ]:
data = {"Group": [], "Dev Time": [], "Burst #": []}

for group in groups:
    for exp in group.experiments.values():
        for emb_n, emb in enumerate(exp.embryos.values()):
            trace = emb.trace
            for i, t in zip(range(num_episodes), trace.peak_times):
                data["Group"].append(group.name)
                data["Dev Time"].append(emb.get_DT_from_time(t))
                data["Burst #"].append(i+1)

data = pd.DataFrame(data)

myplots.plot_pointplot(data, x="Burst #", y="Dev Time", category="Group", rc=episode_metrics_rc)

## CDF Dev Time Per Burst

In [ ]:
data = {"Group": [], "Dev Time": []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos.values():
            trace = emb.trace
            dev_times = [emb.get_DT_from_time(t) for t in trace.peak_times]
            data["Dev Time"].extend(dev_times)
            data["Group"].extend([group.name] * len(dev_times))

data = pd.DataFrame(data)

myplots.plot_cdf(data, "Dev Time", category="Group", rc=group_metrics_rc)

# Other Features

## Burst Amplitude

In [ ]:
data = {"Group": [], "Amplitude": [], "Burst #": []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos.values():
            t = emb.trace
            for i, amp in zip(range(num_episodes), t.peak_amplitudes):
                data["Amplitude"].append(amp)
                data["Group"].append(group.name)
                data["Burst #"].append(i)

data = pd.DataFrame(data)

myplots.plot_pointplot(data, x="Burst #", y="Amplitude", category="Group", rc=episode_metrics_rc)

## AUC Bins

In [ ]:
data = {"Group": [], "AUC": [], "Bin": []}

n_bins = 5
first_bin = 2
bin_width = 0.2

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos.values():
            trace = emb.trace
            dev_time_at_peaks = emb.get_DT_from_time(trace.peak_times)
            bins = [first_bin + j * bin_width for j in range(n_bins)]
            bin_idxs = utils.split_in_bins(dev_time_at_peaks, bins)
            data["Group"].extend([str(group.name)] * len(trace.peak_aucs))
            data["AUC"].extend(trace.peak_aucs)
            data["Bin"].extend(bin_idxs)

data = pd.DataFrame(data)

# make x labels
bins.append(first_bin + bin_width * n_bins)
x_labels = [f"{s}~{e}" for (s, e) in zip(bins[:-1], bins[1:])]

myplots.plot_pointplot(data, x="Bin", y="AUC", category="Group", rc=episode_metrics_rc, xlabels=x_labels)

## Local Peaks Per Burst

In [ ]:
data = {"Group": [], "# Local Peaks": [], "Burst #": []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos.values():
            trace = emb.trace
            local_peaks = trace.compute_local_peaks(height=0.03, prominence=0.02)
            for i, lp in zip(range(15), local_peaks):
                data["# Local Peaks"].append(lp)
                data["Burst #"].append(i+1)
                data["Group"].append(group.name)

data = pd.DataFrame(data)

myplots.plot_pointplot(data, x="Burst #", y="# Local Peaks", category="Group", rc=episode_metrics_rc)

## Burst Rates, Grouped by Bins

In [ ]:
data = {"Group": [], "Rate": [], "Bin": []}

n_bins = 40
first_bin = 1
bin_width = 0.2
bins = np.arange(start=first_bin, stop=first_bin + n_bins * bin_width, step=bin_width)

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos.values():
            trace = emb.trace
            time_bins, idx_offset = emb.get_time_bins(bins)
            time_intervals = np.diff(time_bins) / 3600
            time_bin_idxs = utils.split_in_bins(trace.peak_times, time_bins) - 1
            freqs = Counter(time_bin_idxs)
            rates = {k + idx_offset: v / time_intervals[k] for k, v in freqs.items()}
            data["Group"].extend([str(group.name)] * len(rates))
            data["Rate"].extend(rates.values())
            data["Bin"].extend(rates.keys())

data = pd.DataFrame(data)

# make x labels
first_used_bin = data["Bin"].min()
last_used_bin = data["Bin"].max()
values_to_add = list(range(first_used_bin, last_used_bin))
missing_values = [val for val in values_to_add if val not in data["Bin"].values]
if missing_values:
    data = pd.concat([data, pd.DataFrame({"Bin": missing_values})])
x_labels = [
    round(bins[first_used_bin] + bin_width * i, 1)
    for i in range(last_used_bin - first_used_bin + 1)
]

myplots.plot_pointplot(data, x="Bin", y="Rate", category="Group", rc=episode_metrics_rc, xlabels=x_labels, lines=False)

## Burst Intervals, Grouped by Bins

In [ ]:
data = {"Group": [], "Interval": [], "Bin": []}

n_bins = 40
first_bin = 1
bin_width = 0.2
bins = [first_bin + j * bin_width for j in range(n_bins)]

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos.values():
            trace = emb.trace
            lin_dev_time = emb.lin_developmental_time()
            # skip the last peak because when we calculate peak intervals we loose a peak
            dev_time_at_peaks = lin_dev_time[trace.peak_idxes[:-1]]
            bin_idxs = utils.split_in_bins(dev_time_at_peaks, bins)
            data["Group"].extend([str(group.name)] * len(trace.peak_intervals))
            data["Interval"].extend(trace.peak_intervals / 60)
            data["Bin"].extend(bin_idxs)

data = pd.DataFrame(data)

# make x labels
first_used_bin = data["Bin"].min()
last_used_bin = data["Bin"].max()
values_to_add = list(range(first_used_bin, last_used_bin))
missing_values = [val for val in values_to_add if val not in data["Bin"].values]
if missing_values:
    data = pd.concat([data, pd.DataFrame({"Bin": missing_values})])
x_labels = [
    round(bins[first_used_bin] + bin_width * i, 1)
    for i in range(last_used_bin - first_used_bin + 1)
]

myplots.plot_pointplot(data, x="Bin", y="Interval", category="Group", rc=episode_metrics_rc, xlabels=x_labels, lines=False)